In [2]:
#importamos las librerías necesarias para trabajar
import nltk
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import json
import pandas as pd
import ast
import matplotlib.pyplot as plt
from textblob import TextBlob
import pyarrow

In [3]:
#cargamos el archivo json
with open('C:\\Users\\Tom\\Desktop\\PIMLOps-STEAM\\dataset\\australian_users_items.json', 'r', encoding='utf-8') as file:
    data = [eval(line) for line in file]

#convertimos en un dataframe el archivo y lo desanidamos completamente
user_items = pd.DataFrame(data)
user_items_explored = user_items.explode('items')
user_items_normalized = pd.json_normalize(user_items_explored['items'])
user_items_explored.reset_index(drop=True, inplace=True)
user_items_normalized.reset_index(drop=True, inplace=True)
user_items_final = pd.concat([user_items_explored.drop('items',axis=1), user_items_normalized], axis=1)

In [4]:
#aquí renombramos las columnas para que sea vean más prolijas y esteticas
user_items_final.rename(columns={'user_id': 'User ID'}, inplace=True)
user_items_final.rename(columns={'items_count': 'Items Count'}, inplace=True)
user_items_final.rename(columns={'steam_id': 'Steam ID'}, inplace=True)
user_items_final.rename(columns={'user_url': 'User URL'}, inplace=True)
user_items_final.rename(columns={'item_id': 'Item ID'}, inplace=True)
user_items_final.rename(columns={'item_name': 'Item Name'}, inplace=True)
user_items_final.rename(columns={'playtime_forever': 'Playtime Forever'}, inplace=True)
user_items_final.rename(columns={'playtime_2weeks': 'Playtime 2 Weeks'}, inplace=True)

In [5]:
#eliminamos datos nulos de la columna "Item Name"
user_items_final.dropna(subset="Item Name", inplace=True)

In [6]:
#eliminamos datos duplicados del dataframe
user_items_final.drop_duplicates(inplace=True)

In [7]:
#eliminamos las siguientes columnas las cuales no utilizaremos (esto más que nada por cuestiones de peso del archivo)
user_items_final.drop(columns="User URL", inplace=True)
user_items_final.drop(columns="Playtime 2 Weeks", inplace=True)
user_items_final.drop(columns="Items Count", inplace=True)
user_items_final.drop(columns="Item Name", inplace=True)

In [8]:
#reiniciamos el indice del dataframe
user_items_final = user_items_final.reset_index(drop=True)

In [9]:
#llamamos al dataframe para ver como quedó
user_items_final

,User ID,Steam ID,Item ID,Playtime Forever
0,76561197970982479,76561197970982479,10,6.0
1,76561197970982479,76561197970982479,20,0.0
2,76561197970982479,76561197970982479,30,7.0
3,76561197970982479,76561197970982479,40,0.0
4,76561197970982479,76561197970982479,50,0.0
...,...,...,...,...
5094100,76561198329548331,76561198329548331,346330,0.0
5094101,76561198329548331,76561198329548331,373330,0.0
5094102,76561198329548331,76561198329548331,388490,3.0
5094103,76561198329548331,76561198329548331,521570,4.0


In [10]:
column_data_types = user_items_final['Playtime Forever'].dtypes
print(column_data_types)

float64


In [11]:
#creamos un archivo parquet del dataframe
user_items_final.to_parquet('User_Items.parquet')